In [ ]:
# 训练前制作 ID，保存数据

if 'save_before_train' == phase:
    ID = '{}.{}.{}'.format(idpart['tool.word'], idpart['useStopwords'], idpart['classifier'])
    record_data[ID] = {}
    for ilabel in idpart:
        record_data[ilabel][ID] = idpart[ilabel]

    for labelpart in ['raw', 'format']:
        for ilabel in ['train', 'evaluate', 'all']:
            record_data['time.{}.{}'.format(labelpart, ilabel)][ID] = {} if 'raw' == labelpart else 0.0

#         for timepart, timesvm in record_data['time.raw.model'][ID_SVM].items():
#             record_data['time.raw.model'][ID][timepart] = timesvm + timev[timepart]

In [ ]:
# 实验后处理：显示数据

if 'post-display' == phase:
    # sum of time

    for i in timeDict.keys():
        record_data['time.raw.all'][ID][i] = sum([record_data['time.raw.{}'.format(timepart)][ID][i] for timepart in ['train', 'evaluate']])

    # formation of time

    for timepart in ['train', 'evaluate', 'all']:
            record_data['time.format.{}'.format(timepart)][ID] = timeInFormat(record_data['time.raw.{}'.format(timepart)][ID])

#     print(record_data)
#     tmptime = record_data['time.raw.all'][ID]['h'] * 60 * 60 + record_data['time.raw.all'][ID]['min'] * 60 + \
#                 record_data['time.raw.all'][ID]['sec'] + record_data['time.raw.all'][ID]['ms'] / 1e3 + \
#                 record_data['time.raw.all'][ID]['us'] / 1e6 
#     record_data['score-macro-F1'][ID] = record_data['macro-F1'][ID] / tmptime
#     record_data['score-micro-F1'][ID] = record_data['micro-F1'][ID] / tmptime

    records_df_raw = pd.DataFrame(
        data=record_data, 
        columns=record_labels
    )

    # remove raw time
    records_df_display = records_df_raw.drop(record_labels[-3:], axis=1)

    # use reset_index(drop=True) to use integers instead of ID as the indices
    # (data can be distinguish using each column, so ID is dispensable)
    records_df_display = records_df_display.reset_index(drop=True)

    display(records_df_display)

In [ ]:
# One-hot representation for labels
# 标签独热向量化

if 'one-hot-labels' == phase:
#     csvpath_root = os.path.join(paths['dir.dataroot'], 'data_CSV')

    lb = LabelBinarizer()
    lb.fit(df['train']['class'])

    df_new = {}
    for tpart in ['train', 'test']:
        labels = lb.transform(df[tpart]['class'])
        labelsDf = pd.DataFrame(labels, columns=["class-{}".format(i) for i in range(len(lb.classes_))])
        df_new[tpart] = {}
        df_new[tpart]['y'] = labelsDf
        df_new[tpart]['x'] = df[tpart].drop('class', axis=1)
        df_new[tpart]['all'] = df_new[tpart]['x'].join(df_new[tpart]['y'])
        #save in CSV
#         for subpart in ['x', 'y', 'all']:
#             csvpath = os.path.join(csvpath_root, '{}-cleanLabels{}-{}-{}-{}.csv'.format(tpart, idpart['model.source'], idpart['tool.word'], idpart['model.word'], idpart['model.document']))
#             df_new[tpart][subpart].to_csv(csvpath)

In [ ]:
# DNN train
# 训练分类器：DNN
if 'DNN-train' == phase: 
    COL_OUTCOME = 'class'
    COL_FEATURE = [str(col) for col in list(df['train'].columns) if col != COL_OUTCOME]

    cls2num = {cls:ind for (ind, cls) in enumerate(df['train']['class'].unique())}

    def my_input_fn(dataset):
        # Save dataset in tf format
        feature_cols = {
            str(col): tf.constant(
                df[dataset][int(col)].values#[str(col)].values
            )
            for col in COL_FEATURE
        }
        labels = tf.constant([cls2num[labelname] for labelname in df[dataset][COL_OUTCOME].values])
        # Returns the feature columns and labels in tf format
        return feature_cols, labels

    feature_columns = [tf.contrib.layers.real_valued_column(column_name=str(col)) for col in COL_FEATURE]
    clf = tf.contrib.learn.DNNClassifier(
        feature_columns=feature_columns, 
        hidden_units=[512], 
        n_classes=len(df['train']['class'].unique())
    )

    clf.fit(input_fn=lambda: my_input_fn('train'), steps=2000)

In [ ]:
# DNN evaluate
# 评估分类器：DNN
if 'DNN-evaluate' == phase: 
    X_tensor_test, yt = my_input_fn('test')
    tensorPredCls = list(clf.predict(input_fn=lambda: my_input_fn('test')))
    num2cls = {v:k for (k, v) in cls2num.items()}
    tensorPredClsStr = [num2cls[i] for i in tensorPredCls]
    y_test_true = df['test']['class']

    record_data['accuracy'][ID] = accuracy_score(y_test_true, tensorPredClsStr)
    record_data['macro-F1'][ID] = f1_score(y_test_true, tensorPredClsStr, average='macro')
    record_data['micro-F1'][ID] = f1_score(y_test_true, tensorPredClsStr, average='micro')